In [ ]:
# default_exp client

# Client

> client for global prime pay


this client is able to assist you through the GBP payment flow using just python
feel free to comment and contribute

more info at 'https://doc.gbprimepay.com'

In [ ]:
#export
from nicHelper.wrappers import add_method
from numbers import Number
from typing import Optional
import requests, sentry_sdk, urllib

In [ ]:
#export
from globalPrimePay.helper import Keys, checkSum
from globalPrimePay.exceptions import GBPResponseError

## Helper

In [ ]:
#export
class Helper:
  @staticmethod
  def urlEncode(params):
    return urllib.parse.urlencode(params)

  successMap = {'G' : 'Generate' , 'A' : 'Authorize' , 'S' : 'Settle', 'V' : 'Void', 'D' : 'Decline'}

In [ ]:
Helper.urlEncode({'test':'test','test1':1})

'test=test&test1=1'

## Client

In [ ]:
#export

class Client(Helper):
  '''
    main client for interface with global prime pay
    accept 3 inputs from gbp
    pub: public key
    secret: your secret key
    token: your client token
    
    please refer to the main readme which should describe what those are
  '''
  def __init__(self, pub, secret, token='', endpoint = 'https://api.globalprimepay.com'):
    '''
      input:
        pub: str:: publiceKey from web
        secret: str:: secretKey from web
      docs (https://doc.gbprimepay.com)
    '''
    self.key = Keys(pub,secret)
    self.endpoint = endpoint
    self.token = token
  

## Get token

In [ ]:
#export
@add_method(Client)
def getCardToken(self, cardNumber, expirationMonth, expirationYear, securityCode, name, **kwargs):
  
  body = {
    "rememberCard": True,
    "card": {
      "number": cardNumber,
      "expirationMonth": expirationMonth,
      "expirationYear": expirationYear,
      "securityCode": securityCode,
      "name": name
    }
    ,**kwargs
  }

  headersCharge = {
    'Authorization': self.key.pubPL,
    'Content-Type': 'application/json',
  }
  baseURL = self.endpoint
  url = f'{baseURL}/v2/tokens'
  r:requests.Response = requests.post(url,headers = headersCharge, json = body)
  jsonResponse = r.json()
  if r.status_code >= 400: 
    sentry_sdk.add_breadcrumb(category = 'gbPrimePay', data = {'input':body, 'response':jsonResponse} , level = 'critical')
    raise GBPResponseError(f'error response, input payload is {body}, response from gb is {jsonResponse}')
  return jsonResponse


In [ ]:
#hide

sampleKeys = {
  'pub': "vRvqeoxI1js5CJmURGkiueKtAeGD5rte",
  'secret':  "VWkBurzlZi3dJB78N7jLlztntgDpJSQL",
  'token': 'tok'
}
sampleCard = {
      "cardNumber": "4535017710535741",
      "expirationMonth": "05",
      "expirationYear": "28",
      "securityCode": "184",
      "name": "Watcharagon Phokonwong"
    }

In [ ]:
client = Client(**sampleKeys)
r = client.getCardToken(**sampleCard)
print(r)
token = r['card']['token']
r

{'rememberCard': True, 'resultCode': '00', 'resultMessage': 'Success', 'card': {'name': None, 'number': '453501XXXXXX5741', 'expirationMonth': '05', 'expirationYear': '28', 'securityCode': None, 'token': 'adaa55d8-6aaa-4895-8212-ca986d2b350d', 'cardLocation': 'I', 'cardType': 'VIS'}, 'email': None, 'mobilePhone': None, 'userId': None, 'partnerKey': None}


{'rememberCard': True,
 'resultCode': '00',
 'resultMessage': 'Success',
 'card': {'name': None,
  'number': '453501XXXXXX5741',
  'expirationMonth': '05',
  'expirationYear': '28',
  'securityCode': None,
  'token': 'adaa55d8-6aaa-4895-8212-ca986d2b350d',
  'cardLocation': 'I',
  'cardType': 'VIS'},
 'email': None,
 'mobilePhone': None,
 'userId': None,
 'partnerKey': None}

## Charge Card

In [ ]:
#export
@add_method(Client)
def chargeCard(self, token:str, 
               amount:Number, 
               referenceNo:str, 
               customerName:str, 
               customerEmail:str, 
               detail:str,
               responseUrl:str = "https://responseUrl",
               backgroundUrl:str =  "https://backgroundUrl",
               otp:str = "Y",
               **kwargs)->dict:
  '''
    charge card using 3d security,
    input:
      token:str:: card token (generated using client.getCardToken
      amount:Number:: amount to charge
      referenceNo:str:: reference used to check payment status later, should be uniquely generated
      customerName:str
      customerEmail:str
      detail:str
      responseUrl:str:: redirect url
      backgroundUrl:str:: callback url
      otp:str:: should we force user to input OTP 'Y' or 'N'
      **kwargs: any other input, please see full reference here https://doc.gbprimepay.com/full-payment-3d
  '''
  body = {
    "amount": amount,
    "referenceNo": referenceNo,
    "detail": detail,
    "customerName": customerName,
    "customerEmail": customerEmail,
    "card": {
      "token": token
    },
    "otp": otp,
    "responseUrl": responseUrl,
    "backgroundUrl": backgroundUrl,
    **kwargs
  }
  
  url = f'{self.endpoint}/v2/tokens/charge'
  headers = {
    'Authorization': self.key.secPL,
    'Content-Type': 'application/json',
  }
  r = requests.post(url, json=body, headers = headers)
  jsonResponse = r.json()
  # handle error
  if r.status_code >= 400: 
    sentry_sdk.add_breadcrumb(category = 'gbPrimePay', data = {'input':body, 'response':jsonResponse} , level = 'critical')
    raise GBPResponseError(f'error response, input payload is {body}, response from gb is {jsonResponse}')
  return jsonResponse
    


In [ ]:
client = Client(**sampleKeys)
from random import randint

cardPayload = {
  'token': token,
  'amount': 10,
  'referenceNo': f"TEST123{randint(1000,10000)}",
  'customerName': 'nic',
  'customerEmail': 'email@gmail.com',
  'otp': 'Y',
  'detail': 'this is just a test'
}
r = client.chargeCard(**cardPayload)
print(r)
gbpReferenceNo = r['gbpReferenceNo']

{'customerAddress': None, 'amount': 10, 'referenceNo': 'TEST1231685', 'messageToMerchant': None, 'resultCode': '00', 'customerTelephone': None, 'resultMessage': 'Success', 'customerName': 'nic', 'customerEmail': 'email@gmail.com', 'gbpReferenceNo': 'gbp173512137809', 'merchantDefined5': None, 'detail': 'this is just a test', 'merchantDefined3': None, 'merchantDefined4': None, 'merchantDefined1': None, 'merchantDefined2': None}


## Verify OTP

In [ ]:
#export
@add_method(Client)
def verifyOtp(self, gbpReferenceNo):
  params = {
      'publicKey': self.key.pub,
      'gbpReferenceNo': gbpReferenceNo
  }
  headers3d = {
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  url = self.endpoint + "/v2/tokens/3d_secured"
  r = requests.post(url, params=params, headers = headers3d)
  if r.status_code >= 400: 
    sentry_sdk.add_breadcrumb(category = 'gbPrimePay', data = {'input':params, 'response':r.text} , level = 'critical')
    raise GBPResponseError(f'error response, input payload is {params}, response from gb is {r.text}')
  return r.text

In [ ]:
r = client.verifyOtp(gbpReferenceNo)
r

'<!DOCTYPE html>\n\n<html>\n  <head>\n    <title>GB Prime Pay</title>\n    <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n  </head>\n  <body>\n    <div>Loading...</div>\n    <div>\n      <form action="https://simbank.globalprimepay.com/pay/3d_secure" method="post">\n        <input type="hidden" name="MERID" value="21291800058" />\n        <input type="hidden" name="TERMINALID" value="18060001" />\n        <input type="hidden" name="PAN" value="4535017710535741" />\n        <input type="hidden" name="EXPIRYDATE" value="0528" />\n        <input type="hidden" name="CVV2" value="184" />\n        <input type="hidden" name="INVOICENO" value="2211210051" />\n        <input type="hidden" name="AMOUNT" value="1000" />\n        <input type="hidden" name="POSTURL" value="https://api.globalprimepay.com/web/thanachat_gateway/receive/goback" />\n        <input type="hidden" name="POSTURL2" value="https://api.globalprimepay.com/web/thanachat_ga

In [ ]:
import json
data = "{\"rememberCard\":false,\"card\":{\"number\":\"4535017710535741\",\"expirationYear\":\"28\",\"expirationMonth\":\"05\",\"securityCode\":\"184\",\"name\":\"Watcharagon Phokonwong\"}}"
json.loads(data)

{'rememberCard': False,
 'card': {'number': '4535017710535741',
  'expirationYear': '28',
  'expirationMonth': '05',
  'securityCode': '184',
  'name': 'Watcharagon Phokonwong'}}

In [ ]:
{'customerAddress': None, 'amount': 10, 'referenceNo': 'TEST1235543', 'messageToMerchant': None, 'resultCode': '00', 'customerTelephone': None, 'resultMessage': 'Success', 'customerName': 'nic', 'customerEmail': 'email@gmail.com', 'gbpReferenceNo': 'gbp173512058890', 'merchantDefined5': None, 'detail': 'this is just a test', 'merchantDefined3': None, 'merchantDefined4': None, 'merchantDefined1': None, 'merchantDefined2': None}

{'customerAddress': None,
 'amount': 10,
 'referenceNo': 'TEST1235543',
 'messageToMerchant': None,
 'resultCode': '00',
 'customerTelephone': None,
 'resultMessage': 'Success',
 'customerName': 'nic',
 'customerEmail': 'email@gmail.com',
 'gbpReferenceNo': 'gbp173512058890',
 'merchantDefined5': None,
 'detail': 'this is just a test',
 'merchantDefined3': None,
 'merchantDefined4': None,
 'merchantDefined1': None,
 'merchantDefined2': None}

## QR Payment

In [ ]:
#export
@add_method(Client)
def qrPayment(self, amount:Number, referenceNo:str, *args, **kwargs):
  '''
    pass through variable except token is get from the main class, please see docs at 
    https://doc.gbprimepay.com/qrcash
    currently only amount and referenceNo are required
  '''
  url = f'{self.endpoint}/v3/qrcode/text'
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  body = {
    'token': self.token,
    'amount': amount,
    'referenceNo': referenceNo,
    **kwargs
  }
  r = requests.post(url, data = self.urlEncode(body), headers = headers)
  return r.json()
  

In [ ]:
#hide
from nicHelper.secrets import getSecret
from random import randint
sec = getSecret('globalprime-prod')
# print(sec)


In [ ]:
client = Client( pub=sec['public'], secret = sec['secret'], token = sec['token'],endpoint = 'https://api.gbprimepay.com/', )
customerKey = sec['token']
ref = f'test{randint(100,999)}'

r = client.qrPayment(token = customerKey, amount = 15.00, referenceNo=ref,responseUrl = "https://responseUrl",
               backgroundUrl =  "https://backgroundUrl")
r

{'referenceNo': 'test522',
 'qrcode': '00020101021230830016A00000067701011201150105560068127480218000000221121039125031800000000000test5225303764540515.005802TH5910GBPrimePay6304D681',
 'resultCode': '00',
 'gbpReferenceNo': 'gbp8778184907375',
 'resultMessage': 'Success'}

## Check status

In [ ]:
#export
@add_method(Client)
def checkStatus(self, referenceNo, **kwargs):
  url = f'{self.endpoint}/v1/check_status_txn'
  headers = {
    'content-type': 'application/json',
    'Authorization': self.key.secPL
  }
  body = {'referenceNo': referenceNo }
  r:requests.Response = requests.post(url, headers=headers, json = body)
  if r.status_code >=400: raise Exception(f'error checking status {r.text}')
  return r.json()

In [ ]:
#export
@add_method(Client)
def paymentStatus(self, referenceNo, **kwargs):
  status =  self.checkStatus(referenceNo)['txn']['status']
  print('status is', self.successMap[status])
  return status

In [ ]:
client.checkStatus(ref)

{'resultCode': '00',
 'txn': {'amount': '15.00',
  'referenceNo': 'test522',
  'gbpReferenceNo': 'gbp8778184907375',
  'merchantDefined5': None,
  'merchantDefined3': None,
  'merchantDefined4': None,
  'merchantDefined1': None,
  'status': 'G',
  'paymentType': 'Q',
  'merchantDefined2': None},
 'resultMessage': 'Success'}

In [ ]:
client.paymentStatus(ref)

status is Generate


'G'

# wechat payment

In [ ]:
#export
@add_method(Client)
def wechatPay(self, amount:Number, referenceNo:str, *args, **kwargs):
  '''
    pass through variable except token is get from the main class, please see docs at 
    https://doc.gbprimepay.com/wechat
    currently only amount and referenceNo are required
  '''
  url = f'{self.endpoint}v3/wechat/text'
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  body = {
    'token': self.token,
    'amount': amount,
    'referenceNo': referenceNo,
    **kwargs
  }
  r = requests.post(url, data = self.urlEncode(body), headers = headers)
  return r.json()
  

In [ ]:
#hide
from nicHelper.secrets import getSecret
from random import randint
sec = getSecret('globalprime-prod')
# print(sec)

In [ ]:
client = Client( pub=sec['public'], secret = sec['secret'], token = sec['token'],endpoint = 'https://api.gbprimepay.com/' )
customerKey = sec['token']
ref = f'test{randint(100,999)}'

r = client.wechatPay(
    token = customerKey, 
    amount = 15.00, 
    referenceNo=ref,
    responseUrl = "https://responseUrl",
    backgroundUrl =  "https://backgroundUrl"
    )
r

{'referenceNo': 'test134',
 'resultCode': '00',
 'gbpReferenceNo': 'gbp8778184907379',
 'wechat': 'weixin://wxpay/bizpayurl?pr=eqr4cvAzz',
 'resultMessage': 'Success'}

## Alipay

In [ ]:
#export
@add_method(Client)
def alipay(self, amount:Number, referenceNo:str, *args, **kwargs):
  '''
    pass through variable except token is get from the main class, please see docs at 
    https://doc.gbprimepay.com/wechat
    currently only amount and referenceNo are required
  '''
  url = f'{self.endpoint}v2/alipay'
  headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
  }
  body = {
    'token': self.token,
    'amount': amount,
    'referenceNo': referenceNo,
    **kwargs
  }
  r = requests.post(url, data = self.urlEncode(body), headers = headers)
  return r.text
  

In [ ]:
client = Client( pub=sec['public'], secret = sec['secret'], token = sec['token'],endpoint = 'https://api.gbprimepay.com/', )
customerKey = sec['token']
ref = f'test{randint(100,999)}'

r = client.alipay(
    token = customerKey, 
    amount = '15.00', 
    referenceNo=ref,
    responseUrl = "https://responseUrl",
    backgroundUrl =  "https://backgroundUrl",
    detail='test'
    )
r

'<!DOCTYPE html>\n\n<html>\n<head>\n    <title>GB Prime Pay</title>\n    <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n</head>\n<body>\n<div>Loading...</div>\n<div>\n    <form action="https://paygate.ktc.co.th/ktc/eng/merchandize/payment/payForm.jsp" method="post">\n        <input type="hidden" name="merchantId" value="991304584" />\n        <input type="hidden" name="amount" value="15.00" />\n        <input type="hidden" name="orderRef" value="221121017184" />\n        <input type="hidden" name="currCode" value="764" />\n        <input type="hidden" name="successUrl" value="https://api.gbprimepay.com/alipay/ktc_gateway/success" />\n        <input type="hidden" name="failUrl" value="https://api.gbprimepay.com/alipay/ktc_gateway/fail" />\n        <input type="hidden" name="cancelUrl" value="https://api.gbprimepay.com/alipay/ktc_gateway/cancel" />\n        <input type="hidden" name="payType" value="N" />\n        <input type="hidd